In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString, box, Point
from shapely import wkt
import numpy as np
import swifter
import osmnx as ox

from generator import RoadNetwork
from generator.preprocess import convert_polyline


In [2]:
# normal train porto dataset
df = pd.read_csv("../datasets/trajectories/sf/all_gps_points.csv", sep=";") #../datasets/trajectories/porto/train.csv
# indices = np.random.choice(df.shape[0], int(df.shape[0]/10))
# df = df.iloc[indices, :].reset_index()

In [7]:
df

,POLYLINE,timestamp
0,"LINESTRING (-122.41527 37.7874, -122.40859 37....",[0 60 120 185 240 300]
1,"LINESTRING (-122.39951 37.78881, -122.4032 37....",[0 369 370 371 372 437 494 551 601 661 699]
2,"LINESTRING (-122.41304 37.78711, -122.41499 37...",[0 39 101 159 219 260]
3,"LINESTRING (-122.40998 37.80838, -122.41186 37...",[0 76 130 204 252 311 372 452 501 539]
4,"LINESTRING (-122.40998 37.80839, -122.41113 37...",[0 114 144 246 318 365 426 471]
...,...,...
407148,"LINESTRING (-122.43085 37.77202, -122.42726 37...",[0 65 112 172 233 262 318 384 436 480 579 633 ...
407149,"LINESTRING (-122.40082 37.79218, -122.40131 37...",[0 61 128 180 241 300 368 416]
407150,"LINESTRING (-122.4051 37.78521, -122.40291 37....",[0 60 120 190 244 316 360 431 491 550 615 670 ...
407151,"LINESTRING (-122.38849 37.61609, -122.39863 37...",[0 65 114 255 386 428 542 666 718 789 840 902 ...


In [8]:
#df['POLYLINE'].swifter.apply(wkt.loads) # df = convert_polyline(df, 5)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")
gdf["coords"] = gdf["POLYLINE"].swifter.apply(lambda x: list(x.coords))

TypeError: Input must be valid geometry objects: LINESTRING (-122.41527 37.7874, -122.40859 37.78833, -122.40405 37.78945, -122.40471 37.79305, -122.40521 37.79657, -122.40646 37.79779)

In [4]:
from ast import literal_eval
# porto clipped dataset with min 10 gps points per trajectory
df_c = pd.read_csv("../datasets/trajectories/sf/mapped_id_poly_clipped_corrected.csv", sep=";")
df_c['POLYLINE'] = df_c['POLYLINE'].swifter.apply(wkt.loads)
gdf_c = gpd.GeoDataFrame(df_c, crs='epsg:4326', geometry="POLYLINE")
gdf_c["coords"] = gdf_c["POLYLINE"].swifter.apply(lambda x: list(x.coords))

Pandas Apply:   0%|          | 0/418139 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
print(gdf["coords"].str.len().sum(), gdf_c["coords"].str.len().sum())

In [12]:
network = RoadNetwork()
network.load("../osm_data/porto")

In [ ]:
# Folium Heat Map
from folium import plugins
from folium.plugins import HeatMap
from collections import defaultdict
import branca.colormap

map = folium.Map(location=[41.18049, -8.64599],
                    zoom_start = 8)

xmin, ymin, xmax, ymax = network.bounds
coords = gdf_c.iloc[:, -1].values
print(coords.shape)
# cut down to 1/2 random choice, since full data is too big
indices = np.random.choice(coords.shape[0], int(coords.shape[0]/5))
coords = coords[indices]
# print(reduced_coords.shape)
coords = np.concatenate(coords)
print(coords.shape)

heat_data = [(c[1], c[0]) for c in coords]

steps=20
colormap = branca.colormap.linear.viridis.scale(0, 1).to_step(steps)
colormap.caption = 'Density of GPS points with 10m radius'
gradient_map=defaultdict(dict)s
for i in range(steps):
    gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
colormap.add_to(map)

HeatMap(heat_data, radius=10, gradient=gradient_map).add_to(map)
folium.Rectangle([(ymin, xmin), (ymax, xmax)], color="#FE4A49", weight=4, opacity=1).add_to(map)

# Display the map
#map_porto.save("heatmap_gps_points_porto.html")
map

In [ ]:
"""
Analysis of Road Segment mapping generated by fmm
"""
# visualize trajectories that have outlier speed data - one with negative speed and one with inf speed
# negative speeds = trajectories where the vehicle is standing
df = pd.read_csv("../datasets/trajectories/Porto/road-segment-mapping.csv", sep=";")
df.dropna(inplace=True)
df["speed"] = df["speed"].swifter.apply(literal_eval)
df["speed_mean"] = df["speed"].swifter.apply(np.mean)

In [ ]:
# generate samples
sample_neg = df[df["speed_mean"] < 0].iloc[3]
sample_inf = df[df["speed_mean"] > 1e1].iloc[1]
sample_neg["mgeom"] = wkt.loads(sample_neg["mgeom"])
sample_inf["mgeom"] = wkt.loads(sample_inf["mgeom"])

sample_inf["speed_km"] = np.around(np.array(sample_inf["speed"]) * 111000 * 3.6, decimals=1)
step = int(len(list(sample_inf["mgeom"].coords)) / len(sample_inf["speed"]))
sample_inf

In [ ]:
# plot supspecious examples
m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=13,
    tiles="cartodbdark_matter"
)

data = [(c[1], c[0]) for c in list(sample_inf["mgeom"].coords)]
# d2 = [(c[1], c[0]) for c in list(so["geometry"].coords)]
#speed = df_an["speed"][3]
folium.PolyLine(data, color="green", weight=2.5, opacity=0.5).add_to(m)
#folium.PolyLine(d2, color="red", weight=2.5, opacity=1).add_to(m)
for i, speed in enumerate(sample_inf["speed_km"]):
    folium.Marker(
        location=data[i*7],
        popup=str(speed),
        icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: white">{str(speed)} km/h</div>""")
    ).add_to(m)

m

In [ ]:
network = RoadNetwork()
network.load("../osm_data/porto")

In [ ]:
node = network.gdf_edges[(network.gdf_edges["length"]>800) & (network.gdf_edges["osmid"].str.len() < 12)].iloc[-1]
print(node.dropna().drop(["geometry", "highway_enc", "fid"]))

In [ ]:
# plot osm examples of way and node for prelim section
m = folium.Map(
    location=[41.1488008, -8.6121386],
    zoom_start=13,
    tiles="cartodbpositron"
)
# html = f'''
# id: {node.name}<br>
# latitude: {node.y}<br>
# longitude: {node.x}<br>
# highway: {node.highway}<br>
# '''

# iframe = folium.IFrame(html,
#                        width=100,
#                        height=100)

# popup = folium.Popup(iframe,
#                     show=True,
#                     max_width=100)

# folium.CircleMarker(location=[node.y, node.x],
#                         radius=5,
#                         #fill=True,
#                         #fill_opacity=1,
#                         color="red",
#                         # fill_color="red",
#                         weight=5).add_to(m)

data = [(c[1], c[0]) for c in list(node["geometry"].coords)]
# d2 = [(c[1], c[0]) for c in list(so["geometry"].coords)]
#speed = df_an["speed"][3]
folium.PolyLine(data, color="red", weight=5, opacity=0.8).add_to(m)

m

In [4]:
"""
Generate Sample Image for trajectorie that goes out of network bounds and enters later again (For thesis visualization)
"""

df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")
gdf["coords"] = gdf["POLYLINE"].swifter.apply(lambda x: list(x.coords))
network = RoadNetwork()
network.load("../osm_data/porto")

Pandas Apply:   0%|          | 0/1641024 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/32 [00:00<?, ?it/s]

In [6]:
xmin, ymin, xmax, ymax = network.bounds
out_of_bounds_traj = gdf.cx[xmax:, ymax:]

In [ ]:
# sample = out_of_bounds_traj.iloc[np.random.randint(0, out_of_bounds_traj.shape[0])]
sample = out_of_bounds_traj[out_of_bounds_traj["Unnamed: 0"]==800824]

m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=11,
    tiles="cartodbpositron"
)
# filter points 
in_data, out_data = [[], []], []
bbox = box(*network.bounds)
found_out = False
for c in sample.coords.values[0]:
    point = Point(c[0], c[1])
    if point.within(bbox):
        index = 0 if not found_out else 1
        in_data[index].append((c[1], c[0]))
    else:
        found_out = True
        out_data.append((c[1], c[0]))
data = [(c[1], c[0]) for c in sample.coords]
folium.Rectangle([(ymin, xmin), (ymax, xmax)], color="#FE4A49", weight=1.5, opacity=1, dash_array='10').add_to(m)
for ind in in_data:
    folium.PolyLine(ind, color="#21918c", weight=2.5, opacity=1).add_to(m)
folium.PolyLine(out_data, color="#440154", weight=2.5, opacity=1).add_to(m)
folium.PolyLine([in_data[0][-1], out_data[0]], color="#21918c", weight=2.5, opacity=1).add_to(m)
folium.PolyLine([in_data[1][0], out_data[-1]], color="#21918c", weight=2.5, opacity=1).add_to(m)
print(sample["Unnamed: 0"])
# folium.PolyLine(data, color="red", weight=2.5, opacity=1).add_to(m)
# 1047783 573146 800824

m

In [3]:
# Hannover small analysis
from ast import literal_eval
# porto clipped dataset with min 10 gps points per trajectory
df_c = pd.read_csv("../datasets/trajectories/hanover/mapped_id_poly_clipped.csv", sep=";")
df_c['POLYLINE'] = df_c['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df_c, crs='epsg:4326', geometry="POLYLINE")
gdf["coords"] = gdf["POLYLINE"].swifter.apply(lambda x: list(x.coords))

Pandas Apply:   0%|          | 0/153576 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
# plot supspecious examples
m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=13,
    tiles="cartodbdark_matter"
)

sample = gdf.iloc[100]
data = [(c[1], c[0]) for c in list(sample["coords"])]
# d2 = [(c[1], c[0]) for c in list(so["geometry"].coords)]
#speed = df_an["speed"][3]
folium.PolyLine(data, color="green", weight=2.5, opacity=0.5).add_to(m)

m

In [ ]:
# network = RoadNetwork()
# network.load("../osm_data/hanover")
network = RoadNetwork("Hanover, Germany", network_type="drive", retain_all=False, truncate_by_edge=False)

ox.folium.plot_graph_folium(network.G)

In [14]:
network.save(path="../osm_data/hanover")